In [1]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [2]:
df=pd.read_csv("../../media/data/stroke_detailed.csv")
df

,Patient ID,Patient Name,Age,Gender,Hypertension,Heart Disease,Marital Status,Work Type,Residence Type,Average Glucose Level,...,Alcohol Intake,Physical Activity,Stroke History,Family History of Stroke,Dietary Habits,Stress Levels,Blood Pressure Levels,Cholesterol Levels,Symptoms,Diagnosis
0,18153,Mamooty Khurana,56,Male,0,1,Married,Self-employed,Rural,130.91,...,Social Drinker,Moderate,0,Yes,Vegan,3.48,140/108,"HDL: 68, LDL: 133","Difficulty Speaking, Headache",Stroke
1,62749,Kaira Subramaniam,80,Male,0,0,Single,Self-employed,Urban,183.73,...,Never,Low,0,No,Paleo,1.73,146/91,"HDL: 63, LDL: 70","Loss of Balance, Headache, Dizziness, Confusion",Stroke
2,32145,Dhanush Balan,26,Male,1,1,Married,Never Worked,Rural,189.00,...,Rarely,High,0,Yes,Paleo,7.31,154/97,"HDL: 59, LDL: 95","Seizures, Dizziness",Stroke
3,6154,Ivana Baral,73,Male,0,0,Married,Never Worked,Urban,185.29,...,Frequent Drinker,Moderate,0,No,Paleo,5.35,174/81,"HDL: 70, LDL: 137","Seizures, Blurred Vision, Severe Fatigue, Head...",No Stroke
4,48973,Darshit Jayaraman,51,Male,1,1,Divorced,Self-employed,Urban,177.34,...,Rarely,Low,0,Yes,Pescatarian,6.84,121/95,"HDL: 65, LDL: 68",Difficulty Speaking,Stroke
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,13981,Keya Iyer,88,Female,1,1,Divorced,Self-employed,Urban,160.22,...,Social Drinker,High,0,No,Paleo,1.12,171/92,"HDL: 44, LDL: 153",NaN,No Stroke
14996,87707,Anahita Virk,47,Female,0,0,Married,Private,Urban,107.58,...,Never,Low,1,No,Gluten-Free,1.47,155/71,"HDL: 35, LDL: 183",Difficulty Speaking,No Stroke
14997,33174,Ivana Kaur,35,Male,0,0,Married,Government Job,Rural,134.90,...,Rarely,High,1,No,Paleo,0.51,121/110,"HDL: 57, LDL: 159","Difficulty Speaking, Severe Fatigue, Headache",Stroke
14998,22343,Anvi Mannan,73,Male,0,0,Single,Self-employed,Urban,169.42,...,Never,High,0,Yes,Paleo,1.53,157/74,"HDL: 79, LDL: 91","Severe Fatigue, Numbness, Confusion, Dizziness...",No Stroke


In [3]:
df=df.drop(['Patient ID','Patient Name'],axis=1)

In [4]:
df.describe()

,Age,Hypertension,Heart Disease,Average Glucose Level,Body Mass Index (BMI),Stroke History,Stress Levels
count,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000
mean,54.035667,0.249000,0.502933,129.445209,27.474302,0.500267,5.022694
std,21.063111,0.432448,0.500008,40.487792,7.230201,0.500017,2.873223
min,18.000000,0.000000,0.000000,60.000000,15.010000,0.000000,0.000000
25%,36.000000,0.000000,0.000000,94.517500,21.160000,0.000000,2.540000
50%,54.000000,0.000000,1.000000,128.900000,27.420000,1.000000,5.050000
75%,72.000000,0.000000,1.000000,164.592500,33.720000,1.000000,7.520000
max,90.000000,1.000000,1.000000,200.000000,40.000000,1.000000,10.000000


In [5]:
df[['HDL', 'LDL']] = df["Cholesterol Levels"].str.split(",", expand=True)
df=df.drop('Cholesterol Levels',axis=1)
df.head()
df['HDL'] = df['HDL'].str.split(':', expand=True)[1]
df['LDL'] = df['LDL'].str.split(':', expand=True)[1]
df['HDL'] = pd.to_numeric(df['HDL'])
df['LDL'] = pd.to_numeric(df['LDL'])
df.head()


,Age,Gender,Hypertension,Heart Disease,Marital Status,Work Type,Residence Type,Average Glucose Level,Body Mass Index (BMI),Smoking Status,...,Physical Activity,Stroke History,Family History of Stroke,Dietary Habits,Stress Levels,Blood Pressure Levels,Symptoms,Diagnosis,HDL,LDL
0,56,Male,0,1,Married,Self-employed,Rural,130.91,22.37,Non-smoker,...,Moderate,0,Yes,Vegan,3.48,140/108,"Difficulty Speaking, Headache",Stroke,68,133
1,80,Male,0,0,Single,Self-employed,Urban,183.73,32.57,Non-smoker,...,Low,0,No,Paleo,1.73,146/91,"Loss of Balance, Headache, Dizziness, Confusion",Stroke,63,70
2,26,Male,1,1,Married,Never Worked,Rural,189.00,20.32,Formerly Smoked,...,High,0,Yes,Paleo,7.31,154/97,"Seizures, Dizziness",Stroke,59,95
3,73,Male,0,0,Married,Never Worked,Urban,185.29,27.50,Non-smoker,...,Moderate,0,No,Paleo,5.35,174/81,"Seizures, Blurred Vision, Severe Fatigue, Head...",No Stroke,70,137
4,51,Male,1,1,Divorced,Self-employed,Urban,177.34,29.06,Currently Smokes,...,Low,0,Yes,Pescatarian,6.84,121/95,Difficulty Speaking,Stroke,65,68


In [6]:
df['Upper BP'] = df['Blood Pressure Levels'].str.split('/',expand=True)[0]
df['Lower BP'] = df['Blood Pressure Levels'].str.split('/',expand=True)[1]
df['Upper BP'] = pd.to_numeric(df['Upper BP'])
df['Lower BP'] = pd.to_numeric(df['Lower BP'])
df=df.drop('Blood Pressure Levels',axis=1)

In [7]:
df.info()
le = LabelEncoder()
df['Gender'] = le.fit_transform(df['Gender'])
df['Diagnosis']= le.fit_transform(df['Diagnosis'])
df['Marital Status']= le.fit_transform(df['Marital Status'])
df['Residence Type']=le.fit_transform(df['Residence Type'])
df['Family History of Stroke']=le.fit_transform(df['Family History of Stroke'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 22 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age                       15000 non-null  int64  
 1   Gender                    15000 non-null  object 
 2   Hypertension              15000 non-null  int64  
 3   Heart Disease             15000 non-null  int64  
 4   Marital Status            15000 non-null  object 
 5   Work Type                 15000 non-null  object 
 6   Residence Type            15000 non-null  object 
 7   Average Glucose Level     15000 non-null  float64
 8   Body Mass Index (BMI)     15000 non-null  float64
 9   Smoking Status            15000 non-null  object 
 10  Alcohol Intake            15000 non-null  object 
 11  Physical Activity         15000 non-null  object 
 12  Stroke History            15000 non-null  int64  
 13  Family History of Stroke  15000 non-null  object 
 14  Dietar

In [8]:
onehot = OneHotEncoder()
df['Smoking Status']=onehot.fit_transform(df[['Smoking Status']]).toarray()
df['Alcohol Intake']=onehot.fit_transform(df[['Alcohol Intake']]).toarray()
df['Physical Activity']=onehot.fit_transform(df[['Physical Activity']]).toarray()
df['Dietary Habits']=onehot.fit_transform(df[['Dietary Habits']]).toarray()
df['Work Type']=onehot.fit_transform(df[['Work Type']]).toarray()

In [9]:
df.head()

,Age,Gender,Hypertension,Heart Disease,Marital Status,Work Type,Residence Type,Average Glucose Level,Body Mass Index (BMI),Smoking Status,...,Stroke History,Family History of Stroke,Dietary Habits,Stress Levels,Symptoms,Diagnosis,HDL,LDL,Upper BP,Lower BP
0,56,1,0,1,1,0.0,0,130.91,22.37,0.0,...,0,1,0.0,3.48,"Difficulty Speaking, Headache",1,68,133,140,108
1,80,1,0,0,2,0.0,1,183.73,32.57,0.0,...,0,0,0.0,1.73,"Loss of Balance, Headache, Dizziness, Confusion",1,63,70,146,91
2,26,1,1,1,1,0.0,0,189.00,20.32,0.0,...,0,1,0.0,7.31,"Seizures, Dizziness",1,59,95,154,97
3,73,1,0,0,1,0.0,1,185.29,27.50,0.0,...,0,0,0.0,5.35,"Seizures, Blurred Vision, Severe Fatigue, Head...",0,70,137,174,81
4,51,1,1,1,0,0.0,1,177.34,29.06,1.0,...,0,1,0.0,6.84,Difficulty Speaking,1,65,68,121,95


In [10]:
df[df['Symptoms'].isnull()]
df['Symptoms'] = df['Symptoms'].fillna('No Symptoms')



In [11]:
# Extract unique symptoms
unique_symptoms = set()
df['Symptoms'].str.split(', ').apply(unique_symptoms.update)  # Add each symptom to the set

# Display unique symptoms
print(unique_symptoms)

{'Blurred Vision', 'Numbness', 'Weakness', 'No Symptoms', 'Severe Fatigue', 'Loss of Balance', 'Dizziness', 'Confusion', 'Seizures', 'Headache', 'Difficulty Speaking'}


In [12]:
unique_symptoms = pd.DataFrame()


In [13]:
# Create a binary matrix (One-Hot Encoding)
symptoms_matrix = df['Symptoms'].str.get_dummies(sep=', ')

# Display the transformed dataset
print(symptoms_matrix)


       Blurred Vision  Confusion  Difficulty Speaking  Dizziness  Headache  \
0                   0          0                    1          0         1   
1                   0          1                    0          1         1   
2                   0          0                    0          1         0   
3                   1          1                    0          0         1   
4                   0          0                    1          0         0   
...               ...        ...                  ...        ...       ...   
14995               0          0                    0          0         0   
14996               0          0                    1          0         0   
14997               0          0                    1          0         1   
14998               0          1                    0          1         0   
14999               0          0                    0          0         1   

       Loss of Balance  No Symptoms  Numbness  Seizures  Severe

In [14]:
# Concatenate with original dataset
df = pd.concat([df, symptoms_matrix], axis=1)
df.drop(columns=['Symptoms'], inplace=True)

df.head()

,Age,Gender,Hypertension,Heart Disease,Marital Status,Work Type,Residence Type,Average Glucose Level,Body Mass Index (BMI),Smoking Status,...,Confusion,Difficulty Speaking,Dizziness,Headache,Loss of Balance,No Symptoms,Numbness,Seizures,Severe Fatigue,Weakness
0,56,1,0,1,1,0.0,0,0.0,0.0,22.37,...,0,1,0,1,0,0,0,0,0,0
1,80,1,0,0,2,0.0,1,0.0,0.0,32.57,...,1,0,1,1,1,0,0,0,0,0
2,26,1,1,1,1,0.0,0,1.0,0.0,20.32,...,0,0,1,0,0,0,0,1,0,0
3,73,1,0,0,1,0.0,1,1.0,0.0,27.50,...,1,0,0,1,0,0,0,1,1,0
4,51,1,1,1,0,0.0,1,0.0,0.0,29.06,...,0,1,0,0,0,0,0,0,0,0


In [15]:
X=df.drop(columns=['Diagnosis'])
y=df['Diagnosis']
y.value_counts()

Diagnosis
0    7532
1    7468
Name: count, dtype: int64

In [16]:
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

from sklearn.preprocessing import StandardScaler
import pandas as pd

# Assuming X and y are already defined
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Convert scaled data and labels back to DataFrame
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)
y_df = pd.DataFrame(y, columns=["Diagnosis"])

# Combine features and label
final_df = pd.concat([X_scaled_df, y_df], axis=1)

# Save to CSV
final_df.to_csv("../../media/processed/full_data_stroke_detailed.csv", index=False)


In [17]:
# column_names = X.columns
# column_names
#
# # # Combine scaled features and labels for training data
# X_train_df = pd.DataFrame(X_train_scaled, columns=column_names)
# y_train_df = pd.DataFrame(y_train, columns=["Diagnosis"])
# train_combined = pd.concat([X_train_df, y_train_df], axis=1)
#
# # Combine scaled features and labels for test data
# X_test_df = pd.DataFrame(X_test_scaled, columns=column_names)
# y_test_df = pd.DataFrame(y_test, columns=["Diagnosis"])
# test_combined = pd.concat([X_test_df, y_test_df], axis=1)
#
# # # Save combined datasets to CSV
# train_combined.to_csv("../../media/processed/train_data_stroke_detailed.csv", index=False)
# test_combined.to_csv("../../media/processed/test_data_stroke_detailed.csv", index=False)
#
# # Assuming train_combined and test_combined are your datasets
# # combined_df = pd.concat([train_combined, test_combined], ignore_index=True)
#
# # Save the full dataset into one CSV
# # combined_df.to_csv("../../media/processed_datasets/full_data_stroke_detailed.csv", index=False)